In [53]:
%load_ext autoreload
%autoreload 2
import torch.nn as nn
import pandas as pd
from module.Weight_tune import *
from module.Reorg import *
from module.Cram import *
from module.Init import *
from module.LTS import *
from module.Data import *
import datetime
# from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# New Learning mechanism

### Data and Init Model

In [54]:
# INITIALISE MODEL WITH TRAINING TWO LAYER NET
import torch
data = pd.read_csv("Copper_forecasting_data.csv")
X_train, y_train, X_valid, y_valid, X_test, y_test = preprocess(data, new_learning_algorithm=False)
input_dim = X_train.shape[1]
train_loader = loader(X_train, y_train)
valid_loader = loader(X_valid, y_valid)
test_loader = loader(X_test, y_test)

while True:
    model = TwoLayerNet(18, 1, 1).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_losses = []
    val_losses = []   
    val_min_loss = 1000
    for epoch in tqdm(range(100)):
        train_loss = 0
        # forward operation
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)        
            optimizer.zero_grad()
            preds = model(X)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss)
        with torch.no_grad():
            val_loss = 0
            model.eval()
            for X, y in valid_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X)
                loss = criterion(preds, y)
                val_loss += loss.item()
            val_losses.append(val_loss)
            if val_loss < val_min_loss:
                val_min_loss = val_loss
                best_model = model
    print(f"train loss: {criterion(best_model(X_train), y_train)}")
    print(f"val loss: {criterion(best_model(X_valid), y_valid)}")
    if criterion(best_model(X_valid), y_valid) < .001:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

 63%|██████▎   | 63/100 [00:02<00:01, 23.66it/s]


KeyboardInterrupt: 

In [ ]:
data = pd.read_csv("Copper_forecasting_data.csv")
X_train, y_train, X_test, y_test = preprocess(data)
input_dim = X_train.shape[1]
train_loader = loader(X_train, y_train)
test_loader = loader(X_test, y_test)
X_train.device, y_train.device
X_train.shape, y_train.shape, X_test.shape, y_test.shape
np.percentile(((best_model(X_train) - y_train)**2).cpu().detach().numpy(), 95)

0.001186559583611701

In [ ]:
model = best_model

### Setting

In [ ]:
# learning goal: max eps < learning goal
dtype = torch.float64
criterion = nn.MSELoss()
model = best_model
learning_goal = \
    torch.tensor(np.percentile(((model(X_train) - y_train)**2).cpu().detach().numpy(), 95))\
    .to(dtype = dtype).to(device)
lr_rate = .0001
lr_bound = 1e-8
epochs = 50
batch_size = 100
"""
# Note
1. hidden dim should check the previous model. dynamically change
2. For lr_rate, lr bound, lr goal (eps bound) are all the same fro eahc module
"""

# config for weight tune
config_wt = {
    "epochs": 10,
    "criterion": criterion,        # loss function
    "lr_rate": lr_rate,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
}

# config for cram
config_cram = {
    "lr_goal": learning_goal, 
    "s": 0.001,                     # a small num in cram
}

# config for reorganise
config_reorg  = {
    "epochs": 10,
    "criterion": criterion,        # loss function
    "lr_rate": 0.00001,            # learning rate 
    "lr_bound": lr_bound,          # lower bound of learning rate 
    "lr_goal": learning_goal,      # if regular eps < eps_reg: accept the model
    "print_reg": False,            # print detail, eg. loss for each epoch, or not
    "print_w_tune": False,         # print detail, eg. loss for each epoch, or not
    "validate_run": False,         # validate the model, or not
}
# NOTE 
# 1. for leaning goals, if first using weightune and no LTS or otherthings, 
# 13 for getting acceptable wt | 10 not acceptable


### Full Path

- L11 p9, third learning mechanism\
`Check: should y add .reshape(-1, 1)? since some warning shows when testing the final model`\
`Check validate section`    
`Check not using test data to train`

In [ ]:
X_train.shape

torch.Size([423, 18])

In [ ]:
#################################################################
# Full step for the learning algorithm mechanism
# NOTE
# 1. model right before reorg always need to be acceptable model
# 2. model after cram and reorg always need to be acceptable 
# 3. check for the above every time after cram and reorg
# 4. the randomness: in cram find r
#################################################################
from module.Cram import *
from module.Reorg import *
from module.Weight_tune import *

# 0. Open log file
file_create_time = str(datetime.datetime.now().date())
out_file = open(f"log/{file_create_time}" + '.txt', 'a')
write(out_file, f"#######################################")
write(out_file, str(datetime.datetime.now()))
write(out_file, f"#######################################")

# 1. initializing_1_ReLU_LR | L11 p2
"""
model = init_model(X_train, y_train)
"""
n = 0
n_not_fit = 1

while n < len(X_train):
    write(out_file, f"######################################")
    write(out_file, f"---------> Start New lts")
    # 2. obtaining_LTS / selecting_LTS
    train_loader, indices, X_train_lts, y_train_lts, n = \
        lts(model, X_train, y_train, learning_goal, n_not_fit, out_file)

    # 3. check learning goal
    acceptable, eps_sqaure, y_pred = acceptable_eps_ypred(train_loader, model, learning_goal)
    if acceptable:
        continue

    torch.save(model, "unacceptable/selecting.pth")

    # store model in acceptable/wt.pth if acceptable
    # store model in unacceptable/wt.pth if not acceptable
    write(out_file, f"---------> Start module_EU_LG_UA Epoch")
    acceptable, model, train_loss_list, test_loss_list = \
        module_weight_EU_LG_UA(model, train_loader, test_loader, out_file, **config_wt)
    
    if acceptable:
        write(out_file, "---------> Start REORG with accpetable wt")   
        write(out_file, f"model after wt: {model}")
        
        # load model in acceptable/wt.pth if wt acceptable
        pre_module = "wt"
        reorg = reorganising(pre_module, train_loader, test_loader, out_file, **config_reorg)
        reorg.reorganising()
        model = reorg.model
        write(out_file, f"model after reorg: {model}")

    else:
        write(out_file, "---------> Start CRAM and REORG with unacceptable wt //////////")

        # load model before wt: unacceptable/selecting.pth if unacceptable after wt
        model = torch.load("unacceptable/selecting.pth")    
        acceptable, eps_square, y_pred = acceptable_eps_ypred(train_loader, model, learning_goal)
        # write(out_file, f"model after wt: {model}")
        write(out_file, f"eps_square (last 10) before cram: {eps_square[-10:].reshape(-1)}")

        # load model in unacceptable/wt.pth if wt not acceptable       
        # store acceptable cram in acceptable/cram.pth 
        cram = cramming(model, X_train[indices], y_train[indices], out_file, **config_cram)            
        cram.cram() 
        model = cram.model
        acceptable, eps_square, y_pred = acceptable_eps_ypred(train_loader, model, learning_goal)
        # bwrite(out_file, f"model after cram: {model}")  
        write(out_file, f"eps_sqaure (last 10) after cram: {eps_square[-10:].reshape(-1)}")
        assert acceptable, "weird cram"

        # load model in acceptable/cram.pth if cram acceptable
        # store acceptable cram in acceptable/cram.pth
        pre_module = "Cram"
        reorg = reorganising(pre_module, train_loader, test_loader, out_file, **config_reorg)
        reorg.reorganising()
        if reorg.model == "fail regularise" or reorg.model == "no acceptable slfn":
            write(out_file, "weird reorg")

        model = reorg.model
        write(out_file, f"model after reorg: {model}")

        # check if acceptable
        acceptable, eps_square, y_pred = acceptable_eps_ypred(train_loader, model, learning_goal)
        assert acceptable, "weird reorg"
            
            
    torch.cuda.empty_cache()
    
torch.save(model, 'result/model.pth')
    

In [ ]:
model_path = 'result/model.pth'
model = torch.load(model_path)
print(f"train loss: {criterion(model(X_train), y_train)}")
print(f"test loss: {criterion(model(X_test), y_test)}")
print(f'train residual max {torch.max((model(X_train) - y_train)**2)}')
print(f'test residual max {torch.max((model(X_test) - y_test)**2)}')

train loss: 0.00016118382242293717
test loss: 0.0004964259004033544
train residual max 0.0011850889065952751
test residual max 0.005007919463483838


### Benchmark

In [ ]:
data = pd.read_csv("Copper_forecasting_data.csv")
X_train, y_train, X_valid, y_valid, X_test, y_test = preprocess(data, new_learning_algorithm=False)
input_dim = X_train.shape[1]
train_loader = loader(X_train, y_train)
valid_loader = loader(X_valid, y_valid)
test_loader = loader(X_test, y_test)
X_train.device, y_train.device
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

(torch.Size([380, 18]),
 torch.Size([380, 1]),
 torch.Size([43, 18]),
 torch.Size([43, 1]),
 torch.Size([47, 18]),
 torch.Size([47, 1]))

In [ ]:
import matplotlib.pyplot as plt
import torch
# Define the neural network class
class MultiLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MultiLayerNet, self).__init__()
        self.layers = nn.ModuleList()
        
        # Append layers to the list
        for i, hid in enumerate(hidden_size):
            self.layers.append(nn.Linear(input_size, hid).to(dtype=dtype))  # Append linear layer
            if i+1 != len(hidden_size):  # Skip last layer
                self.layers.append(nn.ReLU())  # Append ReLU activation after each linear layer
            input_size = hid  # Update input dimension for the next layer

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
def train_multiple_layer(hidden_sizes):
    while True:
        model = MultiLayerNet(18, hidden_sizes, 1).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        train_losses = []
        val_losses = []   
        val_min_loss = 1000
        for epoch in tqdm(range(500)):
            train_loss = 0
            # forward operation
            for X, y in train_loader:
                X, y = X.to(device), y.to(device)      
                optimizer.zero_grad()
                preds = model(X)
                loss = criterion(preds, y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            train_losses.append(train_loss)
            with torch.no_grad():
                val_loss = 0
                for X, y in valid_loader:
                    X, y = X.to(device), y.to(device)
                    preds = model(X)
                    loss = criterion(preds, y)
                    val_loss += loss.item()
                val_losses.append(val_loss)
                if val_loss < val_min_loss:
                    val_min_loss = val_loss
                    best_model = model
        print(f"train loss: {criterion(best_model(X_train), y_train)}")
        print(f"test loss: {criterion(best_model(X_test), y_test)}")
        if criterion(best_model(X_valid), y_valid) < .001:
            break
    print('Final result ----------------')
    print(f"train loss: {criterion(best_model(X_train), y_train)}")
    print(f"test loss: {criterion(best_model(X_test), y_test)}")
    print(f'train residual max {torch.max((best_model(X_train) - y_train)**2)}')
    print(f'test residual max {torch.max((best_model(X_test) - y_test)**2)}')
    plt.plot(train_losses)
    plt.plot(val_losses)
   

#### Two layer

In [ ]:
train_multiple_layer([100])

#### Multi layer

In [ ]:
train_multiple_layer([100, 100])

#### Four layer

In [58]:
train_multiple_layer([100, 200, 100])

 66%|██████▌   | 328/500 [00:13<00:06, 26.25it/s]

#### Five layer

In [ ]:
train_multiple_layer([100, 200, 200, 100])